## Wrangle Zillow

This notebook will contain my process for acquiring and preparing the Zillow data.
***

### Acquire

In [1]:
#get imports
import numpy as np
import pandas as pd
import env
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
#Write sql query for zillow data
zillow_query = """
    SELECT prop.*,
       pred.logerror,
       pred.transactiondate,
       air.airconditioningdesc,
       arch.architecturalstyledesc,
       build.buildingclassdesc,
       heat.heatingorsystemdesc,
       landuse.propertylandusedesc,
       story.storydesc,
       construct.typeconstructiondesc
FROM   properties_2017 prop
       INNER JOIN (SELECT parcelid,
                   Max(transactiondate) transactiondate
                   FROM   predictions_2017
                   GROUP  BY parcelid) pred
               USING (parcelid)
               			JOIN predictions_2017 as pred USING (parcelid, transactiondate)
       LEFT JOIN airconditioningtype air USING (airconditioningtypeid)
       LEFT JOIN architecturalstyletype arch USING (architecturalstyletypeid)
       LEFT JOIN buildingclasstype build USING (buildingclasstypeid)
       LEFT JOIN heatingorsystemtype heat USING (heatingorsystemtypeid)
       LEFT JOIN propertylandusetype landuse USING (propertylandusetypeid)
       LEFT JOIN storytype story USING (storytypeid)
       LEFT JOIN typeconstructiontype construct USING (typeconstructiontypeid)
WHERE  prop.latitude IS NOT NULL
       AND prop.longitude IS NOT NULL
"""

In [3]:
#Write zillow url to access the codeup database
zillow_url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/zillow'

In [4]:
#Retrieve the information
zillow = pd.read_sql(zillow_query, zillow_url)
zillow.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1727539,14297519,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,6.059063e+13,0.025595,2017-01-01,None,None,None,None,Single Family Residential,None,None
1,1387261,17052889,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,6.111001e+13,0.055619,2017-01-01,None,None,None,None,Single Family Residential,None,None
2,11677,14186244,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,6.059022e+13,0.005383,2017-01-01,None,None,None,None,Single Family Residential,None,None
3,2288172,12177905,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,6.037300e+13,-0.103410,2017-01-01,None,None,None,Central,Single Family Residential,None,None
4,1970746,10887214,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,6.037124e+13,0.006940,2017-01-01,Central,None,None,Central,Condominium,None,None


In [5]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            77381 non-null  int64  
 1   parcelid                      77381 non-null  int64  
 2   airconditioningtypeid         24953 non-null  float64
 3   architecturalstyletypeid      206 non-null    float64
 4   basementsqft                  50 non-null     float64
 5   bathroomcnt                   77381 non-null  float64
 6   bedroomcnt                    77381 non-null  float64
 7   buildingclasstypeid           15 non-null     float64
 8   buildingqualitytypeid         49672 non-null  float64
 9   calculatedbathnbr             76772 non-null  float64
 10  decktypeid                    614 non-null    float64
 11  finishedfloor1squarefeet      6023 non-null   float64
 12  calculatedfinishedsquarefeet  77185 non-null  float64
 13  f

### Prepare

In [6]:
def handle_missing_values(df, prop_required_column = .5, prop_required_row = .70):
    #function that will drop rows or columns based on the percent of values that are missing:\
    #handle_missing_values(df, prop_required_column, prop_required_row
    threshold = int(round(prop_required_column*len(df.index),0))
    df = df.dropna(axis=1, thresh=threshold)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df

In [7]:
# Restrict df to only properties that meet single use criteria
single_use = [261, 262, 263, 264, 266, 268, 273, 276, 279]
zillow = zillow[zillow.propertylandusetypeid.isin(single_use)]

In [8]:
#Now remove things that don't make sense and/or are impossible/illegal.
#If something doesn't sound like the average 'single family residential' property, drop it.
zillow = zillow[(zillow.bedroomcnt > 0) & (zillow.bathroomcnt > 0)]
zillow = zillow[zillow.calculatedfinishedsquarefeet <= 8000]

#There are minimum size laws for single unit homes. 
#Although these change from state to state and county to county,
#A good rule of thumb is 120sqft per bedroom.
zillow = zillow[zillow.calculatedfinishedsquarefeet >= (120 * zillow.bedroomcnt)]

#If the tax amount owed is outrageous, then its probably either wrong or an outlier.
zillow = zillow[zillow.taxamount <= 20_000]

In [9]:
#Also check for properties that are priced significantly higher than normal.
zillow = zillow[zillow.taxvaluedollarcnt < 5_000_000]

In [10]:
#Now handle missing values.
zillow = handle_missing_values(zillow)

/Users/johnathonsmith/opt/miniconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [11]:
# Add column for counties
zillow['county'] = zillow['fips'].apply(
    lambda x: 'Los Angeles' if x == 6037\
    else 'Orange' if x == 6059\
    else 'Ventura')

In [12]:
# drop unnecessary columns
dropcols = ['parcelid',
     'id',
     'calculatedbathnbr',
     'finishedsquarefeet12',
     'fullbathcnt',
     'heatingorsystemtypeid',
     'propertycountylandusecode',
     'propertylandusetypeid',
     'propertyzoningdesc',
     'censustractandblock',
     'propertylandusedesc',
     'rawcensustractandblock',
     'unitcnt',
     'transactiondate',
     'assessmentyear']

zillow = zillow.drop(columns = dropcols)

In [13]:
# assume that since this is Southern CA, null means 'None' for heating system
zillow.heatingorsystemdesc.fillna('None', inplace = True)

In [14]:
# replace nulls with median values for select columns
zillow.lotsizesquarefeet.fillna(7313, inplace = True)
zillow.buildingqualitytypeid.fillna(6.0, inplace = True)

In [15]:
# Just to be sure we caught all nulls, drop them here
zillow = zillow.dropna()

In [16]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67783 entries, 0 to 77380
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bathroomcnt                   67783 non-null  float64
 1   bedroomcnt                    67783 non-null  float64
 2   buildingqualitytypeid         67783 non-null  float64
 3   calculatedfinishedsquarefeet  67783 non-null  float64
 4   fips                          67783 non-null  float64
 5   latitude                      67783 non-null  float64
 6   longitude                     67783 non-null  float64
 7   lotsizesquarefeet             67783 non-null  float64
 8   regionidcity                  67783 non-null  float64
 9   regionidcounty                67783 non-null  float64
 10  regionidzip                   67783 non-null  float64
 11  roomcnt                       67783 non-null  float64
 12  yearbuilt                     67783 non-null  float64
 13  s

In [17]:
zillow.isna().sum()

bathroomcnt                     0
bedroomcnt                      0
buildingqualitytypeid           0
calculatedfinishedsquarefeet    0
fips                            0
latitude                        0
longitude                       0
lotsizesquarefeet               0
regionidcity                    0
regionidcounty                  0
regionidzip                     0
roomcnt                         0
yearbuilt                       0
structuretaxvaluedollarcnt      0
taxvaluedollarcnt               0
landtaxvaluedollarcnt           0
taxamount                       0
logerror                        0
heatingorsystemdesc             0
county                          0
dtype: int64

In [18]:
#Create a few new features and remove features that are related.
zillow['age'] = 2017 - zillow.yearbuilt

In [19]:
#acres: lotsizesquarefeet/43560
zillow['acres'] = zillow.lotsizesquarefeet / 43_560

In [20]:
#tax_rate: taxamount/taxvaluedollarcnt fields (total, land & structure). 
#We can then remove taxamount and taxvaluedollarcnt, 
#and will keep taxrate, tructuretaxvaluedollarcnt, and landtaxvalue.
zillow['tax_rate'] = zillow.taxamount / zillow.taxvaluedollarcnt

In [21]:
# dollar per square foot-structure
zillow['structure_dollar_per_sqft'] = zillow.structuretaxvaluedollarcnt/zillow.calculatedfinishedsquarefeet

In [22]:
# dollar per square foot - land
zillow['land_dollar_per_sqft'] = zillow.landtaxvaluedollarcnt/zillow.lotsizesquarefeet

In [23]:
# ratio of bathrooms to bedrooms
zillow['bath_bed_ratio'] = zillow.bathroomcnt/zillow.bedroomcnt

In [24]:
#Now drop all of the related variables
zillow = zillow.drop(columns = ['taxamount',
                                'taxvaluedollarcnt',
                                'yearbuilt',
                                'fips',
                                'bedroomcnt',
                                'structuretaxvaluedollarcnt',
                                'landtaxvaluedollarcnt',
                                'lotsizesquarefeet',
                                'regionidzip',
                                'regionidcounty',
                                'regionidcity'])

In [25]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67783 entries, 0 to 77380
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bathroomcnt                   67783 non-null  float64
 1   buildingqualitytypeid         67783 non-null  float64
 2   calculatedfinishedsquarefeet  67783 non-null  float64
 3   latitude                      67783 non-null  float64
 4   longitude                     67783 non-null  float64
 5   roomcnt                       67783 non-null  float64
 6   logerror                      67783 non-null  float64
 7   heatingorsystemdesc           67783 non-null  object 
 8   county                        67783 non-null  object 
 9   age                           67783 non-null  float64
 10  acres                         67783 non-null  float64
 11  tax_rate                      67783 non-null  float64
 12  structure_dollar_per_sqft     67783 non-null  float64
 13  l

In [26]:
#Rename columns
zillow.rename(columns = 
             {'bathroomcnt':'bathroom_count',
              'buildingqualitytypeid':'quality_type',
              'calculatedfinishedsquarefeet':'home_square_feet',
              'roomcnt':'room_count',
              'heatingorsystemdesc':'heating_system_desc'}, inplace = True)

In [27]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67783 entries, 0 to 77380
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   bathroom_count             67783 non-null  float64
 1   quality_type               67783 non-null  float64
 2   home_square_feet           67783 non-null  float64
 3   latitude                   67783 non-null  float64
 4   longitude                  67783 non-null  float64
 5   room_count                 67783 non-null  float64
 6   logerror                   67783 non-null  float64
 7   heating_system_desc        67783 non-null  object 
 8   county                     67783 non-null  object 
 9   age                        67783 non-null  float64
 10  acres                      67783 non-null  float64
 11  tax_rate                   67783 non-null  float64
 12  structure_dollar_per_sqft  67783 non-null  float64
 13  land_dollar_per_sqft       67783 non-null  flo

In [28]:
def train_validate_test_split(df, seed = 123):
    '''
    This function takes in a dataframe and an integer for setting a seed
    and splits the data into train, validate and test. 
    Test is 20% of the original dataset, validate is .30*.80= 24% of the 
    original dataset, and train is .70*.80= 56% of the original dataset. 
    The function returns, in this order, train, validate and test dataframes. 
    '''

    train_validate, test = train_test_split(df, test_size=0.2, 
                                            random_state=seed)
    
    train, validate = train_test_split(train_validate, test_size=0.3, 
                                       random_state=seed)
    return train, validate, test

### Train, validate, test splits

I will create two sets: One for exploration and another for modeling.

In [29]:
#For exploring, I can just do a standard split, since I won't need to scale or create dummies
train_exp, validate_exp, test_exp = train_validate_test_split(zillow.copy())

In [30]:
train_exp.shape, validate_exp.shape, test_exp.shape

((37958, 15), (16268, 15), (13557, 15))

In [31]:
#For modeling, I will need to create dummy variables
def get_dummy_vars_and_split(df):
    #Get cols to create dummies for
    cat_cols = df.select_dtypes('object').columns
    
    df_dummies = pd.get_dummies(df[cat_cols], dummy_na=False, drop_first=True)
    df = pd.concat([df, df_dummies], axis = 1).drop(columns = cat_cols)

    train, validate, test = train_validate_test_split(df)

    return train, validate, test

In [32]:
#Feed the train_exp, validate_exp, and test_exp sets into this function as copies
train, validate, test = get_dummy_vars_and_split(zillow.copy())
train.shape, validate.shape, test.shape

((37958, 24), (16268, 24), (13557, 24))

In [33]:
train_exp.head()

,bathroom_count,quality_type,home_square_feet,latitude,longitude,room_count,logerror,heating_system_desc,county,age,acres,tax_rate,structure_dollar_per_sqft,land_dollar_per_sqft,bath_bed_ratio
2508,2.0,6.0,1817.0,34545099.0,-118113299.0,0.0,-0.043667,Central,Los Angeles,32.0,0.171832,0.012560,76.943864,4.669339,0.666667
8820,1.0,6.0,1032.0,33823082.0,-118172928.0,0.0,-0.094636,Central,Los Angeles,88.0,0.104775,0.011991,151.632752,58.403155,0.500000
62841,2.0,6.0,1263.0,33785610.0,-117874268.0,7.0,0.039590,None,Orange,62.0,0.138659,0.011516,52.703880,27.313576,0.666667
46826,2.0,8.0,1804.0,34665318.0,-118072402.0,0.0,0.002990,Central,Los Angeles,9.0,0.175528,0.019511,92.405765,7.363327,0.666667
10027,2.0,6.0,1723.0,34306711.0,-118416238.0,0.0,-0.074936,Central,Los Angeles,55.0,0.172314,0.012442,77.190946,41.833200,0.500000


In [34]:
train.head()

,bathroom_count,quality_type,home_square_feet,latitude,longitude,room_count,logerror,age,acres,tax_rate,...,heating_system_desc_Floor/Wall,heating_system_desc_Forced air,heating_system_desc_Gravity,heating_system_desc_Heat Pump,heating_system_desc_None,heating_system_desc_Radiant,heating_system_desc_Solar,heating_system_desc_Yes,county_Orange,county_Ventura
2508,2.0,6.0,1817.0,34545099.0,-118113299.0,0.0,-0.043667,32.0,0.171832,0.012560,...,0,0,0,0,0,0,0,0,0,0
8820,1.0,6.0,1032.0,33823082.0,-118172928.0,0.0,-0.094636,88.0,0.104775,0.011991,...,0,0,0,0,0,0,0,0,0,0
62841,2.0,6.0,1263.0,33785610.0,-117874268.0,7.0,0.039590,62.0,0.138659,0.011516,...,0,0,0,0,1,0,0,0,1,0
46826,2.0,8.0,1804.0,34665318.0,-118072402.0,0.0,0.002990,9.0,0.175528,0.019511,...,0,0,0,0,0,0,0,0,0,0
10027,2.0,6.0,1723.0,34306711.0,-118416238.0,0.0,-0.074936,55.0,0.172314,0.012442,...,0,0,0,0,0,0,0,0,0,0


## For Testing

In [35]:
import wrangle

In [37]:
zillow = wrangle.wrangle_zillow()

/Users/johnathonsmith/opt/miniconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [38]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67783 entries, 0 to 77380
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   bathroom_count             67783 non-null  float64
 1   quality_type               67783 non-null  float64
 2   home_square_feet           67783 non-null  float64
 3   latitude                   67783 non-null  float64
 4   longitude                  67783 non-null  float64
 5   room_count                 67783 non-null  float64
 6   logerror                   67783 non-null  float64
 7   heating_system_desc        67783 non-null  object 
 8   county                     67783 non-null  object 
 9   age                        67783 non-null  float64
 10  acres                      67783 non-null  float64
 11  tax_rate                   67783 non-null  float64
 12  structure_dollar_per_sqft  67783 non-null  float64
 13  land_dollar_per_sqft       67783 non-null  flo

Good to go.